In [ ]:
import cv2
import numpy as np
import pkg_resources
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
import os
from layer_factory import LayerFactory
from network import Network
from exceptions import InvalidImage

class PNet(Network):
    """
    Network to propose areas with faces.
    """
    def _config(self):
        layer_factory = LayerFactory(self)

        layer_factory.new_feed(name='data', layer_shape=(None, 12, 12, 3))
        layer_factory.new_conv(name='conv1', kernel_size=(3, 3), channels_output=10, stride_size=(1, 1),
                            padding='VALID', relu=False)
        layer_factory.new_prelu(name='prelu1')
        layer_factory.new_max_pool(name='pool1', kernel_size=(2, 2), stride_size=(2, 2))
        layer_factory.new_conv(name='conv2', kernel_size=(3, 3), channels_output=16, stride_size=(1, 1),
                               padding='VALID', relu=False)
        layer_factory.new_prelu(name='prelu2')
        layer_factory.new_conv(name='conv3', kernel_size=(3, 3), channels_output=32, stride_size=(1, 1),
                               padding='VALID', relu=False)
        layer_factory.new_prelu(name='prelu3')
        layer_factory.new_conv(name='conv4-1', kernel_size=(1, 1), channels_output=2, stride_size=(1, 1), relu=False)
        layer_factory.new_softmax(name='prob1', axis=3)

        layer_factory.new_conv(name='conv4-2', kernel_size=(1, 1), channels_output=4, stride_size=(1, 1),
                               input_layer_name='prelu3', relu=False)

    def _feed(self, image):
        print('Running Pnet!')
        return self._session.run(['pnet/conv4-2/BiasAdd:0', 'pnet/prob1:0'], feed_dict={'pnet/input:0': image})

def read_pos_images():
    #Read positive images:
    path, __, filenames = next(os.walk("./pos_train/"))
    file_count = len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[0.0,1.0]]])
    #Create list of coordinates:
    coordinates=[]
    file = open('./coordinates.txt','r')
    lines = file.readlines()
    lines = [line[:-1] for line in lines]
    idx=[1,0,3,2]
    for line in lines:
        line = line.split(" ")
        line = line[1]
        line=line[1:-1]
        line = line.split(",")
        #Transpose coordinates
        x=0
        nline=[]
        for i in idx:
            nline.append(line[i])
            x=x+1
        line=[[[float(c) for c in nline]]]
        coordinates.append(line)
    #Return images, probs, and coordinates
    return images, prob, coordinates

def read_neg_images():
    #Read negative images:
    path, __, filenames = next(os.walk("./neg_train/"))
    file_count = len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[1.0,0.0]]])
    #Create list of coordinates:
    coordinates=[]
    for i in range(file_count):
        coordinates.append([[[0.0,0.0,0.0,0.0]]])
    #Return images, prob, coordinates
    return images, prob, coordinates

#Read in all images, probabilities, and coordinates
pimages, pprob, pcoordinates = read_pos_images()
nimages, nprob, ncoordinates = read_neg_images()
o_images=np.append(pimages,nimages,axis=0)
o_images=np.reshape(o_images,(-1,12,12,3))
o_prob=pprob+nprob
o_coordinates=pcoordinates+ncoordinates

#Shuffle them up using an index
idx=np.arange(len(o_prob))
np.random.shuffle(idx)
images=np.empty_like(o_images)
c=0
for i in idx:
    images[c]=o_images[i]
    c=c+1
images=(images-127.5)/128.0
images = np.transpose(images, (0, 2, 1, 3)) #Transpose images
prob=[]
for i in idx:
    prob.append(o_prob[i])
coordinates=[]
for i in idx:
    coordinates.append(o_coordinates[i])

with tf.Session() as sess:
    with tf.Graph().as_default():
        
        #Initialize training
        sess = tf.Session()
        train_net=PNet(sess)
        bimg=tf.placeholder(tf.float32, shape=(100,12,12,3))
        bprob=tf.placeholder(tf.float32, shape=(100,1,1,2))
        bprobmask=tf.placeholder(tf.float32, shape=(100,1))
        bcoord=tf.placeholder(tf.float32, shape=(100,1,1,4))
        loss=tf.reduce_mean(tf.square(bprob-train_net.get_layer('conv4-1')))+bprobmask*0.5*tf.reduce_mean(tf.square(bcoord-train_net.get_layer('conv4-2')))
        optimizer = tf.train.AdamOptimizer()
        train = optimizer.minimize(loss)
        init = tf.global_variables_initializer()
        sess.run(init)
        saver = tf.train.Saver()
        
        #Test
        img=cv2.imread("1.bmp")
        img1=(img-127.5)/128.0
        img2=np.expand_dims(img1, 0)
        print(train_net.feed(img2))
        
        #Grab a batch of images, probs, and coordinates, and feed into training
        for j in range(10):
            i=0
            f=100
            while f<len(prob):
                batchimg=images[i:f]
                batchprob=prob[i:f]
                k=np.array(prob[i:f])
                k1=np.reshape(k,(100,2))
                k2=k1[:,1]*1.0
                k3=np.reshape(k2,(100,1))
                batchprobmask=k3
                batchcoord=coordinates[i:f]
                i=i+100
                f=f+100
                sess.run(train,feed_dict={'pnet/input:0': batchimg, bprob: batchprob, bcoord: batchcoord,bprobmask:batchprobmask})
            print(train_net.feed(img2))

        tf.trainable_variables()
        wt=np.load('./data/mtcnn_weights.npy')

        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv1/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv1']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv1/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv1']['biases']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/prelu1/alpha:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['prelu1']['alpha']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/prelu2/alpha:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['prelu2']['alpha']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/prelu3/alpha:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['prelu3']['alpha']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv2/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv2']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv2/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv2']['biases']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv3/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv3']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv3/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv3']['biases']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv4-1/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv4-1']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv4-1/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv4-1']['biases']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv4-2/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv4-2']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv4-2/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv4-2']['biases']=w1[0]
        np.save('./data/mtcnn_weights.npy',wt)

In [ ]:
#Testing an image
img=cv2.imread("1.bmp")
img1=(img-127.5)/128.0
img2=np.expand_dims(img1, 0)
print(train_net.feed(img2))

Correct coordinates: 1.bmp: [0.25,0,1,1]